In [ ]:
# --- Setup
import os, sys, importlib, json, time, traceback, math

# (optional) .env
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

os.environ.setdefault("ORS_LOG_LEVEL", "INFO")
if not os.getenv("ORS_API_KEY"):
    raise SystemExit("Set ORS_API_KEY in your environment or .env before running this notebook.")

# Ensure project root on sys.path (this notebook sits in /tests)
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "tests":
    ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, ROOT)

from modules.road.ors_client import ORSConfig, ORSClient
from modules.addressing import resolver
from modules.cabotage import ports_index, router

# Hot reload while iterating
importlib.reload(resolver)
importlib.reload(ports_index)
importlib.reload(router)

cfg = ORSConfig(default_country="BR")
ors = ORSClient(cfg)

def jprint(obj) -> None:
    print(json.dumps(obj, ensure_ascii=False, indent=2, sort_keys=True))


In [ ]:
# --- Door-to-door cabotage (origin CEP → destination CEP)
o = "01310-200"   # Paulista
d = "20010-000"   # Centro RJ

t0 = time.time()
cab = router.find_cabotage_route(
      o
    , d
    , ors=ors
)
dt = (time.time() - t0) * 1000
print(f"\n✓ CABOTAGE in {dt:.0f} ms — result:\n")
jprint(cab)

# Assertions: totals = sum(legs), 2 ports, fields sanity
legs = cab["legs"]
tot  = cab["totals"]

def ssum(key):
    return sum((x[key] or 0.0) for x in legs)

assert abs(tot["distance_km"] - ssum("distance_km")) < 1e-6, "tot.distance != Σ legs"
assert abs(tot["hours"]       - ssum("hours"))       < 1e-6, "tot.hours != Σ legs"
assert abs(tot["cost_brl"]    - ssum("cost_brl"))    < 1e-6, "tot.cost != Σ legs"
assert abs(tot["co2eq_t"]     - ssum("co2eq_t"))     < 1e-6, "tot.co2 != Σ legs"
assert len(cab["ports_used"]) == 2, "expected exactly 2 ports"
assert legs[0]["mode"] == "road" and legs[1]["mode"] == "sea" and legs[2]["mode"] == "road", "mode order mismatch"

print("\nAll cabotage assertions passed ✅")


In [ ]:
# --- Optional sanity: compare full road vs cabotage totals
full_road = ors.route_road(
      "01310-200"
    , "20010-000"
)
full_road.pop("segments", None)

km_road = (full_road["distance_m"] or 0.0) / 1000.0
hrs_road = (full_road["duration_s"] or 0.0) / 3600.0

print("\nFull road (driving-hgv):")
jprint({
      "distance_km": round(km_road, 3)
    , "hours": round(hrs_road, 3)
})

print("\nCabotage totals:")
jprint(cab["totals"])
